# 구현할 코드 구조

![단층퍼셉트론](./image/single_layer_perceptron_CodeTree.PNG)

---

## 0.0 파이썬 모듈 불러오기

In [2]:
import csv
import numpy as np
# import time

np.random.seed(1234)

## 0.1 하이퍼 파라미터 정의

In [3]:
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001 

## A.1 abalone_Exec
- 실험용 메인함수

In [4]:
def abalone_exec(epoch_count=10, mb_size=10, report=1):
    
    load_abalone_dataset() # 데이터 로드
    
    init_model() # 모델 초기화
    
    train_and_test(epoch_count, mb_size, report) # 학습 및 테스트 

## B.1 load_abalone_dataset
- 데이터 적재함수 정의
    
    1. 데이터 불러오기
    2. 전역 변수 생성
    3. 원-핫 인코딩 처리

In [5]:
def load_abalone_dataset():
    with open('./data/abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None) # 데이터의 첫번째 행(칼럼명)을 생략
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt 
    input_cnt, output_cnt = 10, 1 # 입,출력 벡터 크기
    data = np.zeros([len(rows), input_cnt+output_cnt])

    for n, row in enumerate(rows):
        if row[0] == 'I': data[n, 0] = 1 # if문 짧게 쓰는 방법
        if row[0] == 'M': data[n, 1] = 1
        if row[0] == 'F': data[n, 2] = 1
        data[n, 3:] = row[1:]

---
- next() 활용 예제

In [6]:
# next() 사용 O
with open('./data/abalone.csv') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader, None) 
    rows = []
    for row in csvreader:
        rows.append(row)
print(rows[0])
print(len(rows))

['M', '0.455', '0.365', '0.095', '0.514', '0.2245', '0.101', '0.15', '15']
4177


In [7]:
# next() 사용 X
with open('./data/abalone.csv') as csvfile:
    csvreader = csv.reader(csvfile)
#     next(csvreader, None) 
    rows = []
    for row in csvreader:
        rows.append(row)
print(rows[0])
print(len(rows))

['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings']
4178


---

## B.2 init_model
- 파라미터 (랜덤) 초기화 함수

In [8]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
                            # 평균,표준편차,크기
    bias = np.zeros([output_cnt]) # bias는 초반에 영향을 미칠 수 있으므로 처음엔 0으로 지정

## B.3 train_and_test
- 학습 및 평가 함수 정의
    - 하위 함수가 여러 개 있음. 주의.
    
    
    1. 학습
    2. 출력

In [9]:
def train_and_test(epoch_count, mb_size, report):
                   # mb_size : 미니배치크기
                   # report  : 출력 주기
            
    step_count = arrange_data(mb_size) # 미니배치크기에 따른 step_count
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], [] # 손실함수, 정확도 
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n) # mini batch마다 파라미터 업데이트!
            loss, acc = run_train(train_x, train_y)
            losses.append(loss) # step_count 개수만큼의 원소가 생성됨
            accs.append(acc)    # step_count 개수만큼의 원소가 생성됨
                                # loss, accs는 이 후에 평균낼 것
            
        if report > 0 and (epoch+1) % report == 0: # 출력 주기 조건
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
                              # np.mean(accs) : train accracy
                              # acc           : test accuracy
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

---

- arrange_data()의 반환값, 미니배치 스텝 수란

In [10]:
load_abalone_dataset()
print("총 데이터의 수(행)", data.shape[0])

mb_size = 100
step_count = int(data.shape[0] * 0.8) // mb_size  # train data는 전체의 80%
                                                  # 100개씩 데이터를 처리하면 33step이 걸림
print("데이터의 80%의 미니배치 스텝수 :", step_count)
print('train size:',step_count * mb_size)
print(3300/4177)

총 데이터의 수(행) 4177
데이터의 80%의 미니배치 스텝수 : 33
train size: 3300
0.7900406990663156


---

## C.1~3 arrange_data, get_test_data, get_train_data
- 학습 및 평가 데이터 획득 함수 정의

In [11]:
def arrange_data(mb_size): # train: 80%, test: 20%으로 shuffle index
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0]) # m
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size  # integer
    test_begin_idx = step_count * mb_size  # train,test 경계선 index
    return step_count

def get_test_data(): # test set
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:] # X_test, y_text를 return

def get_train_data(mb_size, nth): # train set # nth : minibach 실행 순서
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0: # 각 epoch을 시작할 때, 매번 train set을 선택하는 index를 셔플해 줌
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

## C.4 run_train
- 학습 실행 함수(하나의 step을 수행)

In [12]:
def run_train(x, y):
    # -----순전파 및 정확도 추출
    output, aux_nn = forward_neuralnet(x) # aux_nn : 입력벡터 x
                                          # 역전파에서 사용하려고 저장
    loss, aux_pp = forward_postproc(output, y) # aux_pp : 편차
                                               # 역전파에서 사용하려고 저장
    accuracy = eval_accuracy(output, y)
    
    # -----역전파 과정
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)  # 역전파
    
    return loss, accuracy

## C.5 run_test
- 평가 실행함수 정의

In [13]:
def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

## D.1 & E.1 forward_neuralnet, backprop_neuralnet
- 순전파 및 역전파 함수 정의

In [14]:
def forward_neuralnet(x): # 순전파 계산
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x

def backprop_neuralnet(G_output, x): # 역전파 계산 
                                     # G_output = dL/da
    global weight, bias
    g_output_w = x.transpose() # 각 열에 x(1), x(2), ..., x(m)이 있음
                               # n x m matrix
     
    G_w = np.matmul(g_output_w, G_output) # dw
    G_b = np.sum(G_output, axis=0)        # db

    # 파라미터 업데이트
    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

## D.2 / E.2 후처리 과정에 대한 순전파 및 역전파 함수 정의

In [15]:
def forward_postproc(output, y): # da
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square)
    return loss, diff

def backprop_postproc(G_loss, diff): # ?????????????????노이해
    shape = diff.shape # m
    
    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1

    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff
    
    return G_output

## D.3 정확도 계산 함수 정의  

In [16]:
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output - y)/y))
    return 1 - mdiff

---

## 단층퍼셉트론 메인 함수 실행

In [17]:
abalone_exec()

Epoch 1: loss=33.875, accuracy=0.557/0.812
Epoch 2: loss=8.226, accuracy=0.820/0.814
Epoch 3: loss=7.582, accuracy=0.812/0.809
Epoch 4: loss=7.475, accuracy=0.808/0.811
Epoch 5: loss=7.395, accuracy=0.810/0.809
Epoch 6: loss=7.328, accuracy=0.808/0.810
Epoch 7: loss=7.269, accuracy=0.808/0.811
Epoch 8: loss=7.217, accuracy=0.808/0.812
Epoch 9: loss=7.175, accuracy=0.810/0.810
Epoch 10: loss=7.135, accuracy=0.809/0.810

Final Test: final accuracy = 0.810


## 파라미터 확인

In [18]:
print(weight)
print(bias)

[[1.02697603]
 [1.47450981]
 [1.66960135]
 [2.04468668]
 [1.62513525]
 [0.60292627]
 [2.39993815]
 [0.54107313]
 [0.46878034]
 [1.01969382]]
[4.16894769]


## 새로운 입력 벡터 X에 대한 예측

In [19]:
x = np.array([0,1,0,0.44,0.3,0.08,0.5,0.23,0.11,0.2])
output = forward_neuralnet(x)
print(output)

(array([8.65881481]), array([0.  , 1.  , 0.  , 0.44, 0.3 , 0.08, 0.5 , 0.23, 0.11, 0.2 ]))


## 하이퍼퍼라미터 수정하며 실험

* LEARNING_RATE = 0.001 -> 0.1
* epoch_count = 10 -> 100
* mb_size = 10 -> 100

In [20]:
LEARNING_RATE = 0.1
abalone_exec(epoch_count=100,mb_size=100,report=20)

Epoch 20: loss=5.804, accuracy=0.825/0.831
Epoch 40: loss=5.259, accuracy=0.834/0.828
Epoch 60: loss=5.056, accuracy=0.837/0.838
Epoch 80: loss=4.950, accuracy=0.838/0.840
Epoch 100: loss=4.910, accuracy=0.840/0.826

Final Test: final accuracy = 0.826
